# SpinToolkit

Demo code for Spin wave theory for complex spin orders I: linear spin wave, generalized linear spin wave, multi-magnon continuum and finite-temperature effects_

Authors: Lei Xu, Xiaojian Shi, Yangjie Jiao, and [Zhentao Wang](https://orcid.org/0000-0001-7442-2933)

In this demo, we compute the $T=0$ multi-boson excitation of bilinear-biquadratic model on the triangular lattice.

## Prerequisites

import the modules and set the number of threads for parallelling

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from joblib import Parallel, delayed
import SpinToolkit_py as sptk

In [ ]:
# check the system information and choose the number of threads accordingly
sptk.print_system_info()

n_jobs = 2 # set manually
# n_jobs = n_jobs   # use the maximum number of threads available

## Define lattice and hamiltonian

- triangular lattice with a 1-sublattice supercell compatible with ferroquadrupolar order

- spin-$1$ model with bilinear and biquadratic exchange

- use "SU(N)" mode

In [ ]:
latt = sptk.lattice(name = "triangular", L = [1, 1])
Nsites = latt.Nsites()

In [ ]:
θ = -0.375 * np.pi
J = np.cos(θ)         # nearest-neighbor bilinear exchange
K = np.sin(θ)         # nearest-neighbor biquadratic exchange

hamiltonian = sptk.model_spin(S = 1.0, mode = "SU(N)", lattice = latt)
print()

for site_i in range(Nsites):
    coor_i, sub_i = latt.site2coor(site = site_i)
    coor0_i, Mi   = latt.coor2supercell0(coor = coor_i)
    xi = coor_i[0]
    yi = coor_i[1]

    # 1st neighbor terms
    coor_j      = [xi + 1, yi]
    coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
    site_j      = latt.coor2site(coor = coor_j, sub = 0)
    hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, J),
                                      site_i = site_i, site_j = site_j,
                                      Mi = Mi, Mj = Mj)
    hamiltonian.add_biquadratic(K = K, site_i = site_i, site_j = site_j,
                                Mi = Mi, Mj = Mj)

    coor_j      = [xi, yi + 1]
    coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
    site_j      = latt.coor2site(coor = coor_j, sub = 0)
    hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, J),
                                      site_i = site_i, site_j = site_j,
                                      Mi = Mi, Mj = Mj)
    hamiltonian.add_biquadratic(K = K, site_i = site_i, site_j = site_j,
                                Mi = Mi, Mj = Mj)

    coor_j      = [xi + 1, yi + 1]
    coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
    site_j      = latt.coor2site(coor = coor_j, sub = 0)
    hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, J),
                                      site_i = site_i, site_j = site_j,
                                      Mi = Mi, Mj = Mj)
    hamiltonian.add_biquadratic(K = K, site_i = site_i, site_j = site_j,
                                Mi = Mi, Mj = Mj)

hamiltonian.simplify()
print()
hamiltonian.build_mc_list()
print()

## Perform minimization (minimize from several independent initial random guesses, then pick the lowest-energy solution)

In [ ]:
print("Optimizing ground state", end = "... ", flush = True)
start        = time.perf_counter()
Z_min, f_min = hamiltonian.optimize_spins_SUN(total_seeds = 20) # more seeds -> more likely to hit global minimum
end          = time.perf_counter()
print(f"{end - start}s")
print()

print(f"f_min (global): {f_min}")
for i in range(Nsites):
    print(f"site-{i}: {Z_min[i]}")

## Initialize the GLSW calculation

In [ ]:
hamiltonian.init_GLSW(Z = Z_min)

## Define a $k$-space trajectory, a list of $\omega$ and Gaussian broadening factor $\sigma$ for computing $\mathcal{S}^{ab}({\bm k}, \omega)$

In [ ]:
kc = sptk.k_cut(lattice = latt, density = 50)
kc.add_k(k_new = [0.0, 0.0])              # Gamma
kc.add_k(k_new = [0.5, 0.0])              # M
xtic_M = kc.len_list[-1]                  # x-axis position for M-point (for plotting)
kc.add_k(k_new = [1.0 / 3.0, 1.0 / 3.0])  # K
xtic_K = kc.len_list[-1]                  # x-axis position for K-point (for plotting)
kc.add_k(k_new = [0.0, 0.0])              # Gamma
total_k = kc.size()

ω_max = 20.0
num_ω = 400
ω_list = np.arange(0, ω_max, ω_max / num_ω, dtype = np.float64).tolist()

FWHM  = 0.1
sigma = FWHM / 2.35482

## Compute 1-boson DSSF $\mathcal{S}_{1}^{ab}({\bm k},\omega)$

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

start = time.perf_counter()
for index_k in range(total_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        disp0, Sxx0, Syy0, Szz0, \
        SxyPyx_R0, SyzPzy_R0, SzxPxz_R0, \
        SxyMyx_I0, SyzMzy_I0, SzxMxz_I0 \
        = sptk.DSSF_GLSW(model = hamiltonian, T = 0.0,
                         k = k, omega_list = ω_list,
                         eval_1boson = True, maxeval_2boson = 0,
                         maxeval_3boson = 0, maxeval_0boson = 0,
                         broadening = "Gaussian", sigma_or_eta = sigma,
                         epsilon = 1.0e-4)
        omega_k.append(disp0)
        Sxx.append(Sxx0)
        Syy.append(Syy0)
        Szz.append(Szz0)
        SxyPyx_R.append(SxyPyx_R0)
        SyzPzy_R.append(SyzPzy_R0)
        SzxPxz_R.append(SzxPxz_R0)
        SxyMyx_I.append(SxyMyx_I0)
        SyzMzy_I.append(SyzMzy_I0)
        SzxMxz_I.append(SzxMxz_I0)
    except Exception as e:
        print(f"error: {e}, k = {k}")
print()
end = time.perf_counter()
print(f"Used {end - start:.6f}s")

## Plot 1-boson DSSF $\mathcal{S}_{1}({\bm k}, \omega) \equiv \mathcal{S}_{1}^{xx}({\bm k}, \omega) + \mathcal{S}_{1}^{yy}({\bm k}, \omega) + \mathcal{S}_{1}^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

x = kc.len_list
axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

fig, ax = plt.subplots()

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", vmin = 0, vmax = 100)

xmax = x[-1]
ymax = 9.0
ax.set_xlim(0.0, xmax)
ax.set_ylim(0.0, ymax)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([0, 3, 6, 9])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.56 * xmax, 0.88 * ymax, r'$\mathcal{S}_1(\bm{k},\omega)$', fontsize = 15)
ax.text(0.46 * xmax, 0.79 * ymax, r'$0$', fontsize = 15)
ax.text(0.88 * xmax, 0.79 * ymax, r'$100$', fontsize = 15)

cax = ax.inset_axes([0.52 * xmax, 0.79 * ymax, 0.32 * xmax, 0.05 * ymax], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0, 100])
cbar.set_ticklabels(['', ''])
cbar.ax.tick_params(length = 0)

plt.savefig("Fig_1Boson_DSSF_TL.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute 2-boson DOS $\mathcal{D}_2({\bm k},\omega)$ (should only be used with a primitive super-lattice basis)

In [ ]:
dos       = []
omega_min = []
omega_max = []

start = time.perf_counter()
for index_k in range(total_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        dos0, omega_min0, omega_max0 \
        = sptk.DOS_GLSW(model = hamiltonian,
                        k = k, omega_list = ω_list,
                        maxeval_2boson = 100000, maxeval_3boson = 0,
                        broadening = "Gaussian", sigma_or_eta = sigma,
                        epsilon = 1.0e-4)
        dos.append(dos0)
        omega_min.append(omega_min0)
        omega_max.append(omega_max0)
    except Exception as e:
        print(f"error: {e}, k = {k}")
print()
end = time.perf_counter()
print(f"Used {end - start:.6f}s")

## Plot 2-boson DOS $\mathcal{D}_2({\bm k},\omega)$

In [ ]:
x = kc.len_list
axis_X, axis_Y = np.meshgrid(x, ω_list)

fig, ax = plt.subplots()

cmap = colors.LinearSegmentedColormap.from_list("white_to_gray", [(1, 1, 1), (0.5, 0.5, 0.5)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, np.array(dos).transpose(), cmap = cmap, shading = "auto", vmin = 0, vmax = 2)
ax.plot(x, np.array(omega_min), '--', color = 'orange')
ax.plot(x, np.array(omega_max), '--', color = 'orange')

xmax = x[-1]
ymax = 18.0
ax.set_xlim(0.0, xmax)
ax.set_ylim(0.0, ymax)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([0, 6, 12, 18])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.56 * xmax, 0.88 * ymax, r'$\mathcal{D}_2(\bm{k},\omega)$', fontsize = 15)
ax.text(0.46 * xmax, 0.79 * ymax, r'$0$', fontsize = 15)
ax.text(0.88 * xmax, 0.79 * ymax, r'$2$', fontsize = 15)

cax = ax.inset_axes([0.52 * xmax, 0.79 * ymax, 0.32 * xmax, 0.05 * ymax], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0, 2])
cbar.set_ticklabels(['', ''])
cbar.ax.tick_params(length = 0)

plt.savefig("Fig_2Boson_DOS_TL.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute 2-boson DSSF $\mathcal{S}_{2}^{ab}({\bm k},\omega)$

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

start = time.perf_counter()
for index_k in range(total_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        disp0, Sxx0, Syy0, Szz0, \
        SxyPyx_R0, SyzPzy_R0, SzxPxz_R0, \
        SxyMyx_I0, SyzMzy_I0, SzxMxz_I0 \
        = sptk.DSSF_GLSW(model = hamiltonian, T = 0.0,
                         k = k, omega_list = ω_list,
                         eval_1boson = False, maxeval_2boson = 100000,
                         maxeval_3boson = 0, maxeval_0boson = 0,
                         broadening = "Gaussian", sigma_or_eta = sigma,
                         epsilon = 1.0e-4)
        omega_k.append(disp0)
        Sxx.append(Sxx0)
        Syy.append(Syy0)
        Szz.append(Szz0)
        SxyPyx_R.append(SxyPyx_R0)
        SyzPzy_R.append(SyzPzy_R0)
        SzxPxz_R.append(SzxPxz_R0)
        SxyMyx_I.append(SxyMyx_I0)
        SyzMzy_I.append(SyzMzy_I0)
        SzxMxz_I.append(SzxMxz_I0)
    except Exception as e:
        print(f"error: {e}, k = {k}")
print()
end = time.perf_counter()
print(f"Used {end - start:.6f}s")

## Plot 2-boson DSSF $\mathcal{S}_2({\bm k}, \omega) \equiv \mathcal{S}_2^{xx}({\bm k}, \omega) + \mathcal{S}_2^{yy}({\bm k}, \omega) + \mathcal{S}_2^{zz}({\bm k}, \omega)$

In [ ]:
x = kc.len_list
axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

fig, ax = plt.subplots()

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", vmin = 0, vmax = 2)
# uncomment the following line if you like to overplot the 1-boson dispersion
# ax.plot(x, np.array(omega_k), 'r--')

xmax = x[-1]
ymax = 18.0
ax.set_xlim(0.0, xmax)
ax.set_ylim(0.0, ymax)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([0, 6, 12, 18])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.56 * xmax, 0.88 * ymax, r'$\mathcal{S}_2(\bm{k},\omega)$', fontsize = 15)
ax.text(0.46 * xmax, 0.79 * ymax, r'$0$', fontsize = 15)
ax.text(0.88 * xmax, 0.79 * ymax, r'$2$', fontsize = 15)

cax = ax.inset_axes([0.52 * xmax, 0.79 * ymax, 0.32 * xmax, 0.05 * ymax], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0, 2])
cbar.set_ticklabels(['', ''])
cbar.ax.tick_params(length = 0)

plt.savefig("Fig_2Boson_DSSF_TL.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute 3-magnon DOS $\mathcal{D}_3({\bm k},\omega)$ (should only be used with a primitive super-lattice basis)

In [ ]:
def process_D3(index_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        return sptk.DOS_GLSW(model = hamiltonian,
                             k = k, omega_list = ω_list,
                             maxeval_2boson = 0, maxeval_3boson = 10000000,
                             broadening = "Gaussian", sigma_or_eta = sigma,
                             epsilon = 1.0e-4)
    except Exception as e:
        print(f"error: {e}, k = {k}")

start = time.perf_counter()
results_D3 = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_D3)(index_k) for index_k in range(total_k))
dos, omega_min, omega_max = zip(*results_D3)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

## Plot 3-boson DOS $\mathcal{D}_3({\bm k},\omega)$

In [ ]:
x = kc.len_list
axis_X, axis_Y = np.meshgrid(x, ω_list)

fig, ax = plt.subplots()

cmap = colors.LinearSegmentedColormap.from_list("white_to_gray", [(1, 1, 1), (0.5, 0.5, 0.5)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, np.array(dos).transpose(), cmap = cmap, shading = "auto", vmin = 0, vmax = 2)
ax.plot(x, np.array(omega_min), '--', color = 'orange')
ax.plot(x, np.array(omega_max), '--', color = 'orange')

xmax = x[-1]
ymax = 27.0
ax.set_xlim(0.0, xmax)
ax.set_ylim(0.0, ymax)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([0, 9, 18, 27])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.56 * xmax, 0.88 * ymax, r'$\mathcal{D}_3(\bm{k},\omega)$', fontsize = 15)
ax.text(0.46 * xmax, 0.79 * ymax, r'$0$', fontsize = 15)
ax.text(0.88 * xmax, 0.79 * ymax, r'$2$', fontsize = 15)

cax = ax.inset_axes([0.52 * xmax, 0.79 * ymax, 0.32 * xmax, 0.05 * ymax], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0, 2])
cbar.set_ticklabels(['', ''])
cbar.ax.tick_params(length = 0)

plt.savefig("Fig_3Boson_DOS_TL.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()

## Compute 3-boson DSSF $\mathcal{S}_3^{ab}({\bm k},\omega)$

In [ ]:
def process_S3(index_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        return sptk.DSSF_GLSW(model = hamiltonian, T = 0.0,
                              k = k, omega_list = ω_list,
                              eval_1boson = False, maxeval_2boson = 0,
                              maxeval_3boson = 10000000, maxeval_0boson = 0,
                              broadening = "Gaussian", sigma_or_eta = sigma,
                              epsilon = 1.0e-4)
    except Exception as e:
        print(f"error: {e}, k = {k}")

start = time.perf_counter()
results_S3 = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_S3)(index_k) for index_k in range(total_k))
omega_k, Sxx, Syy, Szz, \
SxyPyx_R, SyzPzy_R, SzxPxz_R, \
SxyMyx_I, SyzMzy_I, SzxMxz_I = zip(*results_S3)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

## Plot 3-boson DSSF $\mathcal{S}_3({\bm k}, \omega) \equiv \mathcal{S}_3^{xx}({\bm k}, \omega) + \mathcal{S}_3^{yy}({\bm k}, \omega) + \mathcal{S}_3^{zz}({\bm k}, \omega)$

In [ ]:
x = kc.len_list
axis_X, axis_Y = np.meshgrid(x, ω_list)

fig, ax = plt.subplots()

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, np.array(dos).transpose(), cmap = cmap, shading = "auto", vmin = 0, vmax = 4)
# uncomment the following line if you like to overplot the 1-boson dispersion
# ax.plot(x, np.array(omega_k), 'r--')

xmax = x[-1]
ymax = 27.0
ax.set_xlim(0.0, xmax)
ax.set_ylim(0.0, ymax)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([0, 9, 18, 27])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(0.56 * xmax, 0.88 * ymax, r'$\mathcal{S}_3(\bm{k},\omega)$', fontsize = 15)
ax.text(0.46 * xmax, 0.79 * ymax, r'$0$', fontsize = 15)
ax.text(0.88 * xmax, 0.79 * ymax, r'$4$', fontsize = 15)

cax = ax.inset_axes([0.52 * xmax, 0.79 * ymax, 0.32 * xmax, 0.05 * ymax], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0, 4])
cbar.set_ticklabels(['', ''])
cbar.ax.tick_params(length = 0)

plt.savefig("Fig_3Boson_DSSF_TL.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()